<a href="https://colab.research.google.com/github/deepraj16/kaggle-work/blob/main/Copy_of_Sickle_cell_classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
florencetushabe_sickle_cell_disease_dataset_path = kagglehub.dataset_download('florencetushabe/sickle-cell-disease-dataset')

print('Data source import complete.')


100%|██████████| 253M/253M [00:05<00:00, 51.8MB/s]

Extracting files...


Data source import complete.


# Introduction

The Sickle Cell Dataset is the very first Uganda Based Data Collected of Its Kind. We are excited to have been among the first to interact with it. This Notebook shows how we trained a model to classify the Images.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
sample_image = "/kaggle/input/sickle-cell-disease-dataset/Positive/Labelled/1.jpg"


In [ ]:
image = plt.imread(sample_image)
print(image.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/sickle-cell-disease-dataset/Positive/Labelled/1.jpg'

In [ ]:
plt.imshow(image)

In [ ]:
neg_image = plt.imread("/kaggle/input/sickle-cell-disease-dataset/Negative/Clear/1.jpg")
plt.imshow(neg_image)

In [ ]:
import os

negative_imgs = len(os.listdir("/kaggle/input/sickle-cell-disease-dataset/Negative/Clear"))
positive_imgs = len(os.listdir("/kaggle/input/sickle-cell-disease-dataset/Positive/Labelled"))

print("total images = ", (negative_imgs + positive_imgs))

In [ ]:
fig, ax = plt.subplots()

fruits = ['positive images', 'negative images']
counts = [positive_imgs, negative_imgs]
bar_labels = ['positive images', 'negative images']
bar_colors = ['tab:red', 'tab:blue']

ax.bar(fruits, counts, label=bar_labels, color=bar_colors)

ax.set_ylabel('Count')
ax.set_title('Sickle Cell Data Distribution')
ax.legend()


In [ ]:
data_df = pd.DataFrame(columns = ["Image dir", "Label"])

In [ ]:
positive_img_dir = "/kaggle/input/sickle-cell-disease-dataset/Positive/Labelled"
negative_img_dir = "/kaggle/input/sickle-cell-disease-dataset/Negative/Clear"

In [ ]:
import os
import re
import cv2

Images = []
Labels = []

#append in the  postive Images
for file_dir, _, files in os.walk(positive_img_dir):
    for file in files:
        img_file = os.path.join(file_dir, file)
        img = cv2.imread(img_file)
        img = cv2.resize(img, (255, 255))
        Images.append(img)
        Labels.append(1)

In [ ]:
#append in the negative Images
for file_dir, _, files in os.walk(negative_img_dir):
    for file in files:
        img_file = os.path.join(file_dir, file)
        img = cv2.imread(img_file)
        img = cv2.resize(img, (255, 255))
        Images.append(img)
        Labels.append(0)

In [ ]:
Images = np.array(Images)

Images.shape

In [ ]:
sample = Images[0]

plt.imshow(sample)
plt.title("Postive")

In [ ]:
sample = Images[-1]
plt.imshow(sample)
plt.title("Negative")

# Train the models

In [ ]:
X = Images / 255 #scale the data
y = Labels

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, np.array(y), stratify = np.array(y),
                                                    random_state = 42,
                                                    shuffle = True, test_size = 0.2)

train_X.shape, test_X.shape, train_y.shape, test_y.shape

In [ ]:
np.unique(train_y, return_counts = True) , np.unique(test_y, return_counts = True)

# Logistic Regression

Wen dealing with the normal ML models, we have to ensure the data is 2 dimensional, so the first step is to reshape the data to a 2d format

In [ ]:
from sklearn.linear_model import LogisticRegression

train_X_flattened = train_X.reshape(len(train_X), -1)
test_X_flattened = test_X.reshape(len(test_X), -1)

In [ ]:
train_X_flattened.shape

In [ ]:
train_y.shape

In [ ]:
lr = LogisticRegression(random_state = 42).fit(train_X_flattened, train_y)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns

In [ ]:
def get_score(y_pred, y_true):
    print("Accuracy score - ", accuracy_score(y_pred, y_true))
    print("Precision score - ", precision_score(y_pred, y_true))
    print("Recall score - ", recall_score(y_pred, y_true))
    print("f1 score - ", f1_score(y_pred, y_true))
    print("----------  Classification Report   -------")
    print(classification_report(y_pred, y_true))

    matrix = confusion_matrix(y_pred, y_true)
    sns.heatmap(matrix, cmap='Blues', annot=True, fmt='d')

In [ ]:
pred = lr.predict(train_X_flattened)

In [ ]:
get_score(pred, train_y)

In [ ]:
pred = lr.predict(test_X_flattened)

In [ ]:
get_score(pred, test_y)

# Ridge Classifier

In [ ]:
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier(random_state = 42).fit(train_X_flattened, train_y)

In [ ]:
get_score(ridge.predict(train_X_flattened), train_y)

In [ ]:
get_score(ridge.predict(test_X_flattened), test_y)

# SVM (Kernel Trick)

In [ ]:
from sklearn.svm import SVC

svc = SVC(random_state = 42).fit(train_X_flattened, train_y)

In [ ]:
get_score(svc.predict(train_X_flattened), train_y)

In [ ]:
get_score(svc.predict(test_X_flattened), test_y)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state = 42).fit(train_X_flattened, train_y)

In [ ]:
get_score(forest.predict(train_X_flattened), train_y)

In [ ]:
get_score(forest.predict(test_X_flattened), test_y)

# Extra Trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

xtree = ExtraTreesClassifier(random_state = 42).fit(train_X_flattened, train_y)

In [ ]:
get_score(xtree.predict(train_X_flattened), train_y)

In [ ]:
get_score(xtree.predict(test_X_flattened), test_y)

# CNN Model

In [ ]:
from tensorflow import keras

In [ ]:
# Importing all necessary libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import plot_model

img_width, img_height = 255, 255
input_shape = (img_width, img_height, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Add a callback to avoid overfitting

model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath = "best_model.keras",
    monitor='val_accuracy',
    save_best_only=True
)

In [ ]:
history = model.fit(train_X, train_y, validation_data = (test_X, test_y),
          epochs = 20, callbacks = [model_checkpoint])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
best_model = keras.models.load_model("/kaggle/working/best_model.keras")

# Test the model

In [ ]:
predictions = best_model.predict(test_X)

In [ ]:
predictions = np.round(predictions)

In [ ]:
y_pred = model.predict(train_X)
y_pred

In [ ]:
y_pred = np.round(y_pred)
get_score(y_pred, train_y)

In [ ]:
get_score(predictions, test_y)

# Model with Torch

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
from PIL import Image
import random
import torchinfo

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
train_X_tensor, train_y_tensor,test_X_tensor,  test_y_tensor = torch.from_numpy(train_X * 255), torch.from_numpy(train_y), torch.from_numpy(test_X * 255), torch.from_numpy(test_y)

train_X_tensor

In [ ]:
class SickleCellDataset(Dataset):
    def __init__(self,tensors, transform = None):
        self.tensors = tensors
        self.transform = transform
        self.classes = ["Negative", "Positive"]
        self.class_to_idx = {"Negative" : 0, "Positive" : 1}

    def load_image(self, index):
        img_tensor = self.tensors[0][index]
        return Image.fromarray(img_tensor.numpy().astype(np.uint8))

    def __getitem__(self, index):
        x = self.load_image(index)

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

# Don't augment test data, only reshape
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
train_dataset = SickleCellDataset(tensors = (train_X_tensor, train_y_tensor),
                                  transform = train_transforms)

test_data = SickleCellDataset(tensors = (test_X_tensor, test_y_tensor),
                             transform = test_transforms)

In [ ]:
len(train_dataset), len(test_data)

In [ ]:
# 1. Take in a Dataset as well as a list of class names
def display_random_images(dataset: torch.utils.data.dataset.Dataset,
                          classes: list = None,
                          n: int = 10,
                          display_shape: bool = True,
                          seed: int = None):

    # 2. Adjust display if n too high
    if n > 10:
        n = 10
        display_shape = False
        print(f"For display purposes, n shouldn't be larger than 10, setting to 10 and removing shape display.")

    # 3. Set random seed
    if seed:
        random.seed(seed)

    # 4. Get random sample indexes
    random_samples_idx = random.sample(range(len(dataset)), k=n)

    # 5. Setup plot
    plt.figure(figsize=(16, 8))

    # 6. Loop through samples and display random samples
    for i, targ_sample in enumerate(random_samples_idx):
        targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

        # 7. Adjust image tensor shape for plotting: [color_channels, height, width] -> [color_channels, height, width]
        targ_image_adjust = targ_image.permute(1, 2, 0)

        # Plot adjusted samples
        plt.subplot(1, n, i+1)
        plt.imshow(targ_image_adjust)
        plt.axis("off")
        if classes:
            title = f"class: {classes[targ_label]}"
            if display_shape:
                title = title + f"\nshape: {targ_image_adjust.shape}"
        plt.title(title)

In [ ]:
# Display random images from ImageFolderCustom Dataset
display_random_images(train_dataset,
                      n=12,
                      classes=train_dataset.classes,
                      seed=None) # Try setting the seed for reproducible images

In [ ]:
# Turn train and test custom Dataset's into DataLoader's
from torch.utils.data import DataLoader
train_dataloader_custom = DataLoader(dataset=train_dataset, # use custom created train Dataset
                                     batch_size=1, # how many samples per batch?
                                     num_workers=2, # how many subprocesses to use for data loading? (higher = more)
                                     shuffle=True) # shuffle the data?

test_dataloader_custom = DataLoader(dataset=test_data, # use custom created test Dataset
                                    batch_size=1,
                                    num_workers=2,
                                    shuffle=False) # don't usually need to shuffle testing data

train_dataloader_custom, test_dataloader_custom

In [ ]:
# Get image and label from custom DataLoader
img_custom, label_custom = next(iter(train_dataloader_custom))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img_custom.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label_custom.shape}")

In [ ]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [ ]:
class DCNNSCD(nn.Module):
    def __init__(self,input_channels : int, output_shape : int):
        super().__init__()
        self.cnn_block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels= input_channels,
                out_channels=32,
                kernel_size = 2,
                padding = 1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2)
        )

        self.cnn_block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels = 32,
                out_channels=64,
                kernel_size = 2,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2)
        )

        self.cnn_block_3 = nn.Sequential(
            nn.Conv2d(
                in_channels = 64,
                out_channels= 64,
                kernel_size = 2,
                stride = 1,
                padding = 1,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        self.output_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=50176, out_features=64),
            nn.Dropout(p=0.5),
            nn.Linear(in_features = 64, out_features=32),
            nn.Linear(in_features=32, out_features=1)
        )

    def forward(self, x):
        x = self.cnn_block_1(x)

        x = self.cnn_block_2(x)

        x = self.cnn_block_3(x)

        x = self.output_block(x)

        return x


model = DCNNSCD(input_channels=3, output_shape=1).to(device)
model

In [ ]:
input_batch, output_batch = next(iter(train_dataloader_custom))

img, label = input_batch[0].unsqueeze(axis = 0), output_batch[0]

img.shape, label

In [ ]:
model.eval()

with torch.inference_mode():
    pred = model(img.to(device))

print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.sigmoid(pred)}\n")
print(f"Output prediction label:\n{torch.round(torch.sigmoid(pred)).squeeze(dim = 1)}\n")
print(f"Actual label:\n{label}")

In [ ]:
torchinfo.summary(model, input_size=[1, 3, 224, 224])

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X).squeeze(dim = 1)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y.float())
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.round(torch.sigmoid(y_pred))
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X).squeeze(dim = 1)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y.float())
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = torch.round(torch.sigmoid(test_pred_logits))
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [ ]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.BCEWithLogitsLoss(),
          epochs: int = 10):

    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)

        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
torch.manual_seed(1234)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_results = train(model=model,
                        train_dataloader=train_dataloader_custom,
                        test_dataloader=test_dataloader_custom,
                        optimizer=optimizer, epochs = 20)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

In [ ]:
def plot_loss_curves(results):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """

    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
plot_loss_curves(model_results)

In [ ]:
torch.save(model.state_dict(), "model state dict.pt")